In [1]:
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool
import numpy as np
import pandas as pd
import make_data
import datetime
import requests
import glob
import sys
import os
import json
import time
import re
import gensim as gs

In [2]:
names = [405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430]

In [3]:
len(names)

48

In [4]:
sm = 0
for i in names:
    f = open('data/categories/{}_urls.txt'.format(i))
    lines = f.readlines()
    sm += len(lines)
    f.close()
print(sm)

7238


In [5]:
tree = np.load('data/tree.npy')

In [6]:
def dfs(tree, parents, childrens):
    mas = []
    if tree['children'] != None:
        for i in tree['children']:
            parents[i['a_attr']['title'].split('.')[0]] = tree['a_attr']['title'].split('.')[0]
            mas.extend([i['a_attr']['title'].split('.')[0]])
            dfs(i, parents, childrens)
    childrens[tree['a_attr']['title'].split('.')[0]] = mas

In [7]:
parents = {'0':'0'}
childrens = {}
for i in tree:
    parents[i['a_attr']['title'].split('.')[0]] = '0'
    dfs(i, parents, childrens)

In [8]:
def getData(tree, names, allUrls, parent_numb, st):
    dataUrls = []
    dataText = []
    dataCategories = []
    
    number = int(tree['a_attr']['title'].split('.')[0])
    if number in names and number in files:
        f = open('data/filterData/categories/{}.txt'.format(number))
        lines = f.readlines()
        categUrls = [i.split(', ', 1)[0] for i in lines]
        categText = [i.split(', ', 1)[1] for i in lines]
        f.close()
        
        for i in range(len(categUrls)):
            if categUrls[i] not in allUrls:
                dataUrls.extend([categUrls[i]])
                dataText.append([categText[i].replace('\n', '')])
                dataCategories.extend([number])

                allUrls.add(categUrls[i])
            
    if len(tree['children']) != 0:
        for i in tree['children']:
            a, b, c = getData(i, names, allUrls, number, st)
            dataUrls.extend(a)
            dataText.extend(b)
            dataCategories.extend(c)
    
    else:
        if number in names and number in files \
            and (len(categUrls) - len(dataUrls)) / len(categUrls) > 0.8:
            dataCategories = (np.ones(len(dataCategories)) * parent_numb).tolist()

    return dataUrls, dataText, dataCategories

In [9]:
files = glob.glob('data/filterData/categories/*')
files = [int(i[27:].split('.')[0]) for i in files]

In [10]:
allUrls = set()
dataUrls = []
dataText = []
dataCategories = []
st = '   '
parent_numb = 1
for i in tree:
    a, b, c = getData(i, names, allUrls, parent_numb, st)
    dataUrls.extend(a)
    dataText.extend(b)
    dataCategories.extend(c)

In [11]:
dataUrls = np.array(dataUrls)
dataText = np.array(dataText,  dtype='O')
dataCategories = np.array(dataCategories)

In [12]:
len(set(dataCategories))

47

In [13]:
len(dataUrls)

6255

In [14]:
dataAns = [(dataUrls[i], dataCategories[i]) for i in range(dataUrls.shape[0])]

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
aaa = list(set(dataCategories))

In [17]:
dataUrls1 = np.array([])
dataText1 = np.array([],  dtype='O')
dataCategories1 = np.array([])
for i in aaa:
    if np.sum(dataCategories == i) > 1:
        dataUrls1 = np.append(dataUrls1, dataUrls[dataCategories == i])
        dataText1 = np.append(dataText1, dataText[dataCategories == i])
        dataCategories1 = np.append(dataCategories1, dataCategories[dataCategories == i])

In [18]:
dataUrls = dataUrls1
dataText = dataText1
dataCategories = dataCategories1

In [19]:
dataAns = [(dataUrls[i], dataCategories[i]) for i in range(dataUrls.shape[0])]

In [32]:
trainText, testText, trainAns, testAns = \
    train_test_split(dataText, dataAns, test_size=0.2, stratify=dataCategories)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier, NearestNeighbors
from sklearn.cross_validation import cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import RidgeClassifier, LinearRegression, LogisticRegression

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
trainText = np.load('data/filterData/patTrainText.npy')
trainAns = np.load('data/filterData/patTrainAns.npy')
testText = np.load('data/filterData/patTestText.npy')
testAns = np.load('data/filterData/patTestAns.npy')

In [3]:
trainCategory = [float(i[1]) for i in trainAns]
testCategory = [float(i[1]) for i in testAns]

In [4]:
words = {}
for i in trainText:
    for j in i[0].split():
        if j in words:
            words[j] += 1
        else:
            words[j] = 1

IndexError: string index out of range

In [8]:
trainTexts = trainText
testTexts = testText

### w2v

In [18]:
trainTokens = np.array([i.split() for i in trainTexts])
testTokens = np.array([i.split() for i in testTexts])

In [19]:
%%time
model_w2v = gs.models.Word2Vec(trainTokens)

CPU times: user 43.2 s, sys: 116 ms, total: 43.3 s
Wall time: 16.2 s


In [47]:
mas = model_w2v.corpus_count

In [30]:
trainFeatures = np.zeros((trainTokens.shape[0], 100))
for i in range(trainTokens.shape[0]):
    for j in trainTokens[i]:
        if j in model_w2v:
            trainFeatures[i] += model_w2v[j]
    trainFeatures[i] /= len(trainTokens[i]) + 1
    
testFeatures = np.zeros((testTokens.shape[0], 100))
for i in range(testTokens.shape[0]):
    for j in testTokens[i]:
        if j in model_w2v:
            testFeatures[i] += model_w2v[j]
    testFeatures[i] /= len(testTokens[i]) + 1

KeyboardInterrupt: 

### tfidf

In [9]:
tfidfTransformer = TfidfVectorizer()
trainFeatures  = tfidfTransformer.fit_transform(trainTexts)
testFeatures = tfidfTransformer.transform(testTexts)

In [10]:
trainFeatures

<5003x81614 sparse matrix of type '<class 'numpy.float64'>'
	with 1633026 stored elements in Compressed Sparse Row format>

In [12]:
np.sum(trainFeatures != 0, axis=1).max()

4381

In [13]:
np.average(np.sum(trainFeatures != 0, axis=1))

326.40935438736756

# CLF

In [42]:
_time = time.time()
clf = LogisticRegression()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  19.248169660568237
0.716227018385
0.7913669064748201


In [43]:
_time = time.time()
clf = RidgeClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  23.949588775634766
0.747402078337
0.8185451638689049


In [44]:
_time = time.time()
clf = NearestCentroid()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.14675235748291016
0.669064748201
0.812150279776179


In [45]:
_time = time.time()
clf = KNeighborsClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.010329723358154297
0.676258992806
0.7553956834532374


In [46]:
_time = time.time()
clf = KNeighborsClassifier(algorithm='brute', metric='cosine')
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.008327960968017578
0.693844924061
0.7777777777777778


In [47]:
_time = time.time()
clf = SVC()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  74.06704688072205
0.116706634692
0.1486810551558753


In [48]:
_time = time.time()
clf = SVC(gamma=0.5)
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  100.61144852638245
0.713029576339
0.8033573141486811
